In [ ]:
%load_ext autoreload
%autoreload 2

In [47]:
import sys;
sys.path.append(r'C:\Monitor\\')
sys.path.append(r'\\ac-geneva-24\E\grains trading\Streamlit\Monitor\\')

from datetime import datetime as dt
from copy import deepcopy

import pandas as pd; pd.options.mode.chained_assignment = None
import Utilities.Modeling as um

import Utilities.GLOBAL as GV
import Utilities.Modeling as um
import Models.Corn_USA_Yield as cy

import plotly.express as px

In [ ]:
prec_units='in'
temp_units='F'

scope = cy.Define_Scope()
raw_data = cy.Get_Data_All_Parallel(scope)

In [ ]:
milestones =cy.Milestone_from_Progress(raw_data)
intervals = cy.Intervals_from_Milestones(milestones)

train_DF_instr = um.Build_DF_Instructions('weighted',GV.WD_HIST, prec_units=prec_units, temp_units=temp_units, ext_mode = GV.EXT_DICT)
train_df = cy.Build_DF(raw_data, milestones, intervals, train_DF_instr)
model = um.Fit_Model(train_df,'Yield',GV.CUR_YEAR)
# print(model.summary())

In [33]:
# Scenarios Creation
year_start = 1985
year_end = 2022

scenarios_EXT_dict = {}
for p in range(year_start,year_end):
    for t in range(year_start,year_end):
        scenarios_EXT_dict[str(p)+'_'+str(t)]={GV.WV_PREC:GV.EXT_ANALOG+'_'+str(p),GV.WV_SDD_30:GV.EXT_ANALOG+'_'+str(t)}

print(len(scenarios_EXT_dict))
# print(scenarios_EXT_dict)

1369


In [ ]:
raw_data['w_w_df_all'][GV.WD_GFS].last_valid_index()

In [ ]:
%%time
results = {}
sce_date = raw_data['w_w_df_all'][GV.WD_GFS].last_valid_index()
i=0
for sce_name, sce_dict in scenarios_EXT_dict.items():
    i+=1; print(i)
    pred_DF_instr=um.Build_DF_Instructions('weighted',GV.WD_H_GFS, prec_units=prec_units, temp_units=temp_units, ext_mode = sce_dict)
    pred_df = cy.Build_Pred_DF(raw_data,milestones,pred_DF_instr,GV.CUR_YEAR,date_start=sce_date, date_end=sce_date)
    yields = model.predict(pred_df[model.params.index]).values

    results[sce_name]=yields[0]

[print(k,'->',v) for k,v in results.items()]
# print(results)
print('All Done')

In [21]:
%%time
sce_results = cy.Analog_scenarios()

Scenarios to compute: 1369
All Done
Wall time: 1min 11s


In [22]:
min(results.values())

144.1662324890548

In [23]:
min(results, key=results.get)

'2013_2012'

In [36]:
len(sce_results)

1369

In [28]:
sce_date = raw_data['w_w_df_all'][GV.WD_GFS].last_valid_index()

In [29]:
sce_dict=scenarios_EXT_dict['2013_2012']
pred_DF_instr=um.Build_DF_Instructions('weighted',GV.WD_H_GFS, prec_units=prec_units, temp_units=temp_units, ext_mode = sce_dict)
pred_df = cy.Build_Pred_DF(raw_data,milestones,pred_DF_instr,GV.CUR_YEAR,date_start=sce_date, date_end=sce_date)
pred_df

,const,Trend,Yield,Planted pct on May 15th,Planting Prec,Planting Prec Squared,Jul Aug Prec,Jul Aug Prec Squared,Prec Interaction,Pollination SDD,Regular SDD
2022-07-06,1.0,2022,NaN,49.0,8.354471,69.797188,5.84194,34.128267,48.806322,169.86734,108.004738


In [30]:
sce_dict=scenarios_EXT_dict['2013_2020']
pred_DF_instr=um.Build_DF_Instructions('weighted',GV.WD_H_GFS, prec_units=prec_units, temp_units=temp_units, ext_mode = sce_dict)
pred_df = cy.Build_Pred_DF(raw_data,milestones,pred_DF_instr,GV.CUR_YEAR,date_start=sce_date, date_end=sce_date)
pred_df

,const,Trend,Yield,Planted pct on May 15th,Planting Prec,Planting Prec Squared,Jul Aug Prec,Jul Aug Prec Squared,Prec Interaction,Pollination SDD,Regular SDD
2022-07-06,1.0,2022,NaN,49.0,8.354471,69.797188,5.84194,34.128267,48.806322,49.108649,80.058674


In [34]:
sce_dict=scenarios_EXT_dict['1987_2012']
pred_DF_instr=um.Build_DF_Instructions('weighted',GV.WD_H_GFS, prec_units=prec_units, temp_units=temp_units, ext_mode = sce_dict)
pred_df = cy.Build_Pred_DF(raw_data,milestones,pred_DF_instr,GV.CUR_YEAR,date_start=sce_date, date_end=sce_date)
pred_df

,const,Trend,Yield,Planted pct on May 15th,Planting Prec,Planting Prec Squared,Jul Aug Prec,Jul Aug Prec Squared,Prec Interaction,Pollination SDD,Regular SDD
2022-07-06,1.0,2022,NaN,49.0,8.354471,69.797188,9.704058,94.168745,81.072274,169.86734,108.004738


In [66]:
heat_map={}

prec = [k.split('_')[0] for k in sce_results.keys()]
temp = [k.split('_')[1] for k in sce_results.keys()]
yield_value = list(sce_results.values())

# print(x[0])
# print(y[0])
# print(z[0])

heat_map_df = pd.DataFrame({'Precipitation':prec,'Max Temperature':temp,'Yield':yield_value})
heat_map_df.to_csv('Results\Analog_Scenarios.csv')

heat_map_df = heat_map_df.pivot_table(index=['Precipitation'], columns=['Max Temperature'], values=['Yield'], aggfunc='mean')
heat_map_df.columns = heat_map_df.columns.droplevel(level=0)
fig = px.imshow(heat_map_df,color_continuous_scale='RdBu')
fig.show('browser')